# Use PyTorch to Predict Hotel Cancellations

Build and train a neural network to predict if a customer will cancel their hotel booking reservation based on data including the booking dates, average daily cost, number of adults/children/babies, duration of stay, etc

**Setup - Import libraries**

In [1]:
import pandas as pd
import numpy as np

## Import and Inspect

The file `'datasets/resort_hotel_bookings.csv'` contains a subset of a [real-world dataset](https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand) containing reservation and cancellation data for a resort hotel. 

Your goal in this project is build and train a neural network to predict if a customer will cancel their hotel booking reservation based on data including the booking dates, average daily cost, number of adults/children/babies, duration of stay, and so forth.

### Load data

Begin by importing the CSV file to a pandas DataFrame named `hotels`.

Preview the first five rows using the `.head()` method.

In [2]:
hotels = pd.read_csv('datasets/resort_hotel_bookings.csv')
hotels.head()

,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,0,342,2015,July,27,1,0,0,2,0.0,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,0,737,2015,July,27,1,0,0,2,0.0,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,0,7,2015,July,27,1,0,1,1,0.0,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,0,13,2015,July,27,1,0,1,1,0.0,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,0,14,2015,July,27,1,0,2,2,0.0,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


### Explore the data types and whether any data is missing.

Use the `.info()` method on the `hotels` DataFrame to inspect the data.

In [3]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40060 entries, 0 to 40059
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   is_canceled                     40060 non-null  int64  
 1   lead_time                       40060 non-null  int64  
 2   arrival_date_year               40060 non-null  int64  
 3   arrival_date_month              40060 non-null  object 
 4   arrival_date_week_number        40060 non-null  int64  
 5   arrival_date_day_of_month       40060 non-null  int64  
 6   stays_in_weekend_nights         40060 non-null  int64  
 7   stays_in_week_nights            40060 non-null  int64  
 8   adults                          40060 non-null  int64  
 9   children                        40060 non-null  float64
 10  babies                          40060 non-null  int64  
 11  meal                            40060 non-null  object 
 12  country                         

What do we notice about the dataset under inspection?

There are 31 columns and 40,060 total observations in our dataset. The majority of columns do not have missing values.

However, we do notice that: 
- the `agent` and `company` columns seem to have missing values that need to be addressed
- the `country` column has a couple of missing values as well

There are a variety of data types represented. To work with a neural network, we'll have to address any non-numeric columns in our data preparation.

### Explore the cancellation column we want to predict.

Use the `.value_counts()` method on the `is_canceled` column to count the number **and** the percentage of overall cancellations. 

In [4]:
hotels['is_canceled'].value_counts()

is_canceled
0    28938
1    11122
Name: count, dtype: int64

What do we notice about the number of cancellations?

The number of cancellations is much lower than the number of non-cancellations (27.8% canceled vs 72.2% did not cancel). 

We'll need to take this imbalance into account when we evaluate our model. For example, a naive model could simply predict every booking will **not be canceled** and achieve a decent accuracy of 72.2%.

The `reservation_status` column tells us if the booking was canceled while also telling us if the customer was a no-show.

We need to be sure to exclude this column from the training set, otherwise this information will be _leaked_ to our model resulting in inaccurate performance. 

First, let's take a quick look at the values in this column.

Use the `.value_counts()` method on the `reservation_status` column to count the number **and** the percentage of overall cancellations. 

In [5]:
hotels['reservation_status'].value_counts()

reservation_status
Check-Out    28938
Canceled     10831
No-Show        291
Name: count, dtype: int64

What do we notice about the reservation_status column?

The number of no-shows is extremely small and consists of only 291 (or 0.7%) of observations in the dataset.

Before diving into building a model, let's continue to explore the dataset. It's important to understand how different columns interact with cancellations to guide our model structure! 

For example, cancellations might be higher in the summer months (June - September) and lower in the winter months (November - January).

Use the `.groupby()` method to group the data by the `arrival_date_month` column and apply the `.mean()` aggregation function on the `is_canceled` column. This will return the percent of reservations cancelled in each month.

Then, use the `.sort_values()` method to sort the percentages from lowest to highest.

In [6]:
hotels.groupby('arrival_date_month')['is_canceled'].mean().sort_values(ascending=False)

arrival_date_month
August       0.334491
June         0.330706
September    0.323681
July         0.314017
April        0.293433
May          0.287721
October      0.275105
February     0.256204
December     0.238293
March        0.228717
November     0.189167
January      0.148199
Name: is_canceled, dtype: float64

What do we notice about the percentage of cancellations by month?

Winter and spring have the lowest cancellation percentages, while summer and fall have the highest. This information can be very useful for our model!

It might be useful to do more exploratory data analysis to gain additional insights about hotel cancellations. For example, additional analysis may help you select better features to train the model on and exclude features that might seem irrelevant. But for now, let's move on to cleaning and preparing the data.

## Data Cleaning and Preparation

In this section, we'll encode categorical data for use in our neural networks.

To get a sense of the categorical data in the dataset, let's start by previewing the first five rows of all columns with `object` datatype.

Create a list named `object_columns` containing only the names of the object columns (except for the reservation status columns). Select those columns from `hotels` and preview the first `5` rows.

In [7]:
object_columns = hotels.select_dtypes(include='object').columns.difference(['reservation_status'])
hotels[object_columns].head()

,arrival_date_month,assigned_room_type,country,customer_type,deposit_type,distribution_channel,market_segment,meal,reservation_status_date,reserved_room_type
0,July,C,PRT,Transient,No Deposit,Direct,Direct,BB,2015-07-01,C
1,July,C,PRT,Transient,No Deposit,Direct,Direct,BB,2015-07-01,C
2,July,C,GBR,Transient,No Deposit,Direct,Direct,BB,2015-07-02,A
3,July,A,GBR,Transient,No Deposit,Corporate,Corporate,BB,2015-07-02,A
4,July,A,GBR,Transient,No Deposit,TA/TO,Online TA,BB,2015-07-03,A


Preview the first five rows subset by the object columns

Here's how we can subset the DataFrame by the object columns and preview the first five rows:

```py
object_columns = ['arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type']
hotels[object_columns].head()
```

Additionally, it might be helpful to explore the categorical data in each object column using the `.value_counts()` method.

</details>

Typically, we don't want to use every column in training. For example, we may want to drop columns with many missing values or columns that are irrelevant to our prediction task.

Drop any columns you don't want to use to train a cancellation model (do not remove the target label column). Feel free to open our Hint to review the columns we chose to drop in our solution.

Note: We don't want to drop the `reservation_status` column from the dataset quite yet because we'll be using this column to train our multiclass neural network.

Here's why we chose these columns:

- `country` - there are many countries that only appear a handful of times in the dataset which may make our model less generalizable and even discriminate against customers based on their country
- `agent` - similar to `country`, there are many agents that only appear a handful of times which may make our model less generalizable (and there are many missing values!)
- `company` - similar to `agent`, there are many companies that only appear a handful of times which may make our model less generalizable (and there are many missing values!)
- `reservation_status_date` - tells us the date of the latest status change of the reservation which shouldn't be helpful and if anything may leak data
- `arrival_date_week_number` - tells us the week of the year which may be too specific and prone to overfitting
- `arrival_date_day_of_month` - tells us the day of the month which may be too specific and prone to overfitting
- `arrival_date_year` - tells us the year of the booking which may not be helpful to predict future years

</details>

In [8]:
drop_columns = ['country', 'agent', 'company', 'reservation_status_date',
                'arrival_date_week_number', 'arrival_date_day_of_month', 'arrival_date_year']

hotels = hotels.drop(labels=drop_columns, axis=1)

In [9]:
hotels.head()

,is_canceled,lead_time,arrival_date_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,market_segment,...,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status
0,0,342,July,0,0,2,0.0,0,BB,Direct,...,C,C,3,No Deposit,0,Transient,0.0,0,0,Check-Out
1,0,737,July,0,0,2,0.0,0,BB,Direct,...,C,C,4,No Deposit,0,Transient,0.0,0,0,Check-Out
2,0,7,July,0,1,1,0.0,0,BB,Direct,...,A,C,0,No Deposit,0,Transient,75.0,0,0,Check-Out
3,0,13,July,0,1,1,0.0,0,BB,Corporate,...,A,A,0,No Deposit,0,Transient,75.0,0,0,Check-Out
4,0,14,July,0,2,2,0.0,0,BB,Online TA,...,A,A,0,No Deposit,0,Transient,98.0,0,1,Check-Out


## Label Encoding

Encode the `meal` column which tells us which type of meal(s) the customer booked: 

- `Undefined` and `SC` correspond to no meal packages
- `BB` corresponds to breakfast only
- `HB` (half board) corresponds to breakfast + lunch or dinner
- `FB` (full board) corresponds to breakfast, lunch, and dinner.

Label encode the `meal` column with a meaningful order (# of meals booked) using the following scheme:

- `Undefined` and `SC` to `0`
- `BB` to `1`
- `HB` to `2`
- `FB` to `3` 

In [10]:
hotels['meal'] = hotels['meal'].replace({'Undefined':0, 'SC':0, 'BB':1, 'HB':2, 'FB':3})

### One-hot encoding categorical columns

Create a list named `one_hot_columns` containing the list of categorical column names (all the remaining categorical columns) to be one-hot encoded using the `pd.get_dummies()` method.

Preview the cleaned `hotels` DataFrame using the `.head()` method.

In [11]:
one_hot_columns = hotels.select_dtypes(include='object').columns.difference(['reservation_status'])

In [12]:
one_hot_columns

Index(['arrival_date_month', 'assigned_room_type', 'customer_type',
       'deposit_type', 'distribution_channel', 'market_segment',
       'reserved_room_type'],
      dtype='object')

In [13]:
hotels = pd.get_dummies(hotels, columns=one_hot_columns, dtype=int)

Note that the cleaned DataFrame now has 67 columns due to the additional columns created using one-hot encoding.

## Create Training and Testing Sets

Convert our dataset into PyTorch tensors and split them into training and testing sets.

### Import the PyTorch libraries and modules. 

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

### Split training and test set

We need to start by separating our training features from the target labels.

Create a list named `train_features` that contains all of the feature names (column names excluding the target variables `is_canceled` and `reservation_status`).

In [15]:
# Remove target columns
remove_cols = ['is_canceled', 'reservation_status']

# Select training features
train_features = [x for x in hotels.columns if x not in remove_cols]

Using the list of training features in `train_features`, create `X` and `y` tensors:

- `X` contains the data values from the `train_features` columns
- `y` contains the binary labels in the `is_canceled` column in `hotels`

Both `X` and `y` should have the float datatype.

**Be sure to set the correct view of `y` using `.view(-1,1)`**

In [16]:
X = torch.tensor(hotels[train_features].values, dtype=torch.float)
y = torch.tensor(hotels['is_canceled'].values, dtype=torch.float).view(-1,1)

In [17]:
hotels[train_features].head()

,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,...,reserved_room_type_A,reserved_room_type_B,reserved_room_type_C,reserved_room_type_D,reserved_room_type_E,reserved_room_type_F,reserved_room_type_G,reserved_room_type_H,reserved_room_type_L,reserved_room_type_P
0,342,0,0,2,0.0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,737,0,0,2,0.0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,7,0,1,1,0.0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,13,0,1,1,0.0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,14,0,2,2,0.0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


Split our data contained in `X` and `y` into training and testing sets.

Import the `train_test_split` module from Scikit-learn's `sklearn.model_selection` library.

Split `X` and `y` using the following scheme:
- Use 80% of the data for the training set `X_train` and `y_train`
- Use 20% of the data for the testing set `X_test` and `y_test`
- Set the random state to `42` to match our solution

Print out the shape of `X_train` and `X_test` to see how many observations and columns are in the training and testing sets.

How many training features does our training set `X_train` have?

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.80,
                                                    test_size=0.20,
                                                    random_state=42) 
print("Training Shape:", X_train.shape)
print("Testing Shape:", X_test.shape)

Training Shape: torch.Size([32048, 65])
Testing Shape: torch.Size([8012, 65])


## Train a Neural Network for Binary Classification

Create a neural network for Binary Classification to predict hotel cancellations.

Set a random seed to `42` using `torch.manual_seed(42)`.

Build the neural network architecture using `nn.Sequential` with the following:
- input layer with `65` nodes (equal to the number of training features)
- first hidden layer with `36` nodes and a ReLU activation
- second hidden layer with `18` nodes and a ReLU activation
- output layer with `1` node and a Sigmoid activation


In [19]:
# Set a random seed 
torch.manual_seed(42)

# Define the model using nn.Sequential
model = nn.Sequential(
    nn.Linear(65, 36), # 68 is the number of input features in X_train
    nn.ReLU(),
    nn.Linear(36, 18),
    nn.ReLU(),
    nn.Linear(18, 1), # one output node for binary classification
    nn.Sigmoid() # sigmoid activation to output probabilities
)

### Define the loss function and optimizer used for training:

- set the **binary cross-entropy** loss function to the variable `loss`
- set the **Adam** optimizer to the variable `optimizer` with a learning rate of `0.005`

In [20]:
# initialize the BCE loss function
loss = nn.BCELoss()

# initialize SGD optimizer, with a learning rate of .001
optimizer = optim.Adam(
    model.parameters(), 
    lr=0.005)

### Build the training loop to train our neural network.

Train the neural network for `1000` epochs.

Keep track of the training performance by printing out the binary cross-entropy loss and accuracy score every `100` epochs.

Before calculating accuracy, convert the model's predicted probabilities to binary labels (as integers) using `0.5` as the threshold.

In [21]:
from sklearn.metrics import accuracy_score
# set the number of epochs to 300
num_epochs = 1000

for epoch in range(num_epochs):
    ## Add forward pass here, keep the variable name predictions ##+
    predictions = model(X_train) 

    ## Compute BCELoss loss here ##
    BCELoss = loss(predictions, y_train)
  
    ## Compute gradients here ##
    BCELoss.backward()

    ## Update weights and biases here ##
    optimizer.step()
   

    ## Reset the gradients for the next iteration here ##
    optimizer.zero_grad()
    
    if (epoch + 1) % 100 == 0:
        predicted_labels = (predictions >= 0.5).int()
        accuracy = accuracy_score(y_train, predicted_labels)
        print(f'Epoch [{epoch+1}/{num_epochs}], BCELoss: {BCELoss.item():.4f}, Accuracy: {accuracy.item():.4f}')

Epoch [100/1000], BCELoss: 0.3978, Accuracy: 0.8235
Epoch [200/1000], BCELoss: 0.3645, Accuracy: 0.8321
Epoch [300/1000], BCELoss: 0.3509, Accuracy: 0.8372
Epoch [400/1000], BCELoss: 0.3418, Accuracy: 0.8429
Epoch [500/1000], BCELoss: 0.3398, Accuracy: 0.8418
Epoch [600/1000], BCELoss: 0.3379, Accuracy: 0.8417
Epoch [700/1000], BCELoss: 0.3262, Accuracy: 0.8457
Epoch [800/1000], BCELoss: 0.3267, Accuracy: 0.8455
Epoch [900/1000], BCELoss: 0.3218, Accuracy: 0.8471
Epoch [1000/1000], BCELoss: 0.3189, Accuracy: 0.8503


### Evaluate the trained neural network on the testing set:

1. Set the model to **evaluation mode**
2. Turn off gradient calculations
3. Generate predicted probabilities on `X_test`. Save the probabilities to the variable `test_predictions`.
4. Convert the predicted probabilities to binary labels using `0.5` as the threshold. Save the labels to the variable `test_predicted_labels`.

In [22]:
model.eval()
with torch.no_grad():
    ## YOUR SOLUTION HERE ##
    test_predictions = model(X_test)
    test_predicted_labels = (test_predictions >= .5).int()

Recall that the number of cancellations is much lower than the number of non-cancellations (27.8% canceled vs 72.2% did not cancel). 

To evaluate our neural network effectively, compute the accuracy, precision, recall, and F1 scores using the `sklearn.metrics` module:

- use the `accuracy_score` function to compute the overall accuracy
- use the `classification_report` function to compute the precision, recall, and F1 scores


In [23]:
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, test_predicted_labels)
report = classification_report(y_test, test_predicted_labels)

# show output - do not modify
print(f'Accuracy: {accuracy.item():.4f}')
print(report)

Accuracy: 0.8362
              precision    recall  f1-score   support

         0.0       0.86      0.92      0.89      5798
         1.0       0.75      0.61      0.67      2214

    accuracy                           0.84      8012
   macro avg       0.81      0.77      0.78      8012
weighted avg       0.83      0.84      0.83      8012



Overall, the model seems to perform reasonably well at predicting hotel cancellations!

The model has an overall accuracy of 83.7%, indicating that 83.7% of our model's predictions are correct.
The precision score tells us that when our model predicts a cancellation, it is correct ~72% of the time.
The recall score tells us that our model captures about 68% of the actual cancellations in our data. 

In future research, we could improve the model by performing a more in-depth analysis of the features and doing a more robust feature selection process (like gathering more features or dropping less useful features). 

Furthermore, we could modify the neural network architecture by changing the number of nodes across the hidden layers, trying out different activation functions and optimizers, adding more hidden layers, or training on additional epochs.

## Train a Neural Network for Multiclass Classification

Let's now extend our binary classification task to multiclass by attempting to also predict customers who **no-showed** within the `reservation_status` column.

If a hotel can accurately predict no-shows, they can reach out ahead of time to customers who are at high risk of not-showing to their reservation.

### Label encode the three categories in the `reservation_status` column:

- **Check-Out** to `2`
- **Canceled** to `1`
- **No-Show** to `0`

In [24]:
hotels['reservation_status'].value_counts()

reservation_status
Check-Out    28938
Canceled     10831
No-Show        291
Name: count, dtype: int64

In [25]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Fit and transform the 'reservation_status' column
hotels['reservation_status'] = le.fit_transform(hotels['reservation_status'])
hotels['reservation_status'].value_counts()

reservation_status
1    28938
0    10831
2      291
Name: count, dtype: int64

In [26]:
# Display all columns
pd.set_option('display.max_columns', None)

### Create the `X` and `y` tensors where:

- `X` contains the data values from the `train_features` columns
- `y` contains the multiclass data values in the `reservation_status` column

Make sure that `y` uses the `long` datatype.

In [27]:
X = torch.tensor(hotels[train_features].values, dtype=torch.float)
y = torch.tensor(hotels['reservation_status'].values, dtype=torch.long)

### Split the `X` and `y` tensors into training and testing splits using the following scheme:
- Use 80% of the data for the training set `X_train` and `y_train`
- Use 20% of the data for the testing set `X_test` and `y_test`
- Set the random state to `42`

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.80,
                                                    test_size=0.20,
                                                    random_state=42) 
print("Training Shape:", X_train.shape)
print("Testing Shape:", X_test.shape)

Training Shape: torch.Size([32048, 65])
Testing Shape: torch.Size([8012, 65])


### Construct the multiclass neural network with the following architecture:

Set a random seed using `torch.manual_seed(42)`.

- input layer with `65` nodes (equal to the number of training features)
- first hidden layer with `65` nodes and a ReLU activation
- second hidden layer with `36` nodes and a ReLU activation
- final output layer with `3` nodes corresponding to each of the categories in `reservation_status`

Save the network to the variable `multiclass_model`.

In [29]:
# Set a random seed 
torch.manual_seed(42)

# Define the model using nn.Sequential
multiclass_model = nn.Sequential(
    nn.Linear(65, 36), # 65 is the number of input features in X_train
    nn.ReLU(),
    nn.Linear(36, 18),
    nn.ReLU(),
    nn.Linear(18, 3) # one output node for binary classification
)

### Define the loss function and optimizer used for multiclass training:
- set the **cross-entropy** loss function for multiclass to the variable `loss`
- set the **Adam** optimizer to the variable `optimizer` with a learning rate of `0.01`

In [30]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(multiclass_model.parameters(), lr=0.01)

### Build the training loop to train our neural network.

1. Train the neural network for `500` epochs.
2. Keep track of the training performance by printing out the cross-entropy loss and accuracy score every `100` epochs.
3. Be sure to convert the output probabilites of the multiclass model to labels using the `torch.argmax()` function.

In [31]:
from sklearn.metrics import accuracy_score, classification_report

# Train the neural network
num_epochs = 500
for epoch in range(num_epochs):
    predictions =multiclass_model(X_train)
    CELoss = loss(predictions, y_train)
    CELoss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if (epoch + 1) % 100 == 0:
        predicted_labels = torch.argmax(predictions, dim=1)
        accuracy = accuracy_score(y_train, predicted_labels)
        print(f'Epoch [{epoch+1}/{num_epochs}], CELoss: {CELoss.item():.4f}, Accuracy: {accuracy.item():.4f}')

Epoch [100/500], CELoss: 0.4143, Accuracy: 0.8245
Epoch [200/500], CELoss: 0.3899, Accuracy: 0.8305
Epoch [300/500], CELoss: 0.3868, Accuracy: 0.8314
Epoch [400/500], CELoss: 0.3815, Accuracy: 0.8335
Epoch [500/500], CELoss: 0.3662, Accuracy: 0.8377


### Evaluate the trained neural network on the testing set:

1. Set the multiclass model to **evaluation mode**
2. Turn off gradient calculations
3. Generate predicted probabilities on `X_test`. Save the predicted probabilities to the variable `multiclass_predictions`.
4. Select the class with the largest predicted probability using the `torch.argmax()` function. Save the predicted classes to the variable `multiclass_predicted_labels`.

In [32]:
multiclass_model.eval()
with torch.no_grad():
    multiclass_predictions = multiclass_model(X_test)
    multiclass_predicted_labels = torch.argmax(multiclass_predictions, dim=1)


### Evaluate the multiclass neural network by calculating the overall accuracy, precision, recall, and F1 scores.

Using the `sklearn.metrics` module:
- use the `accuracy_score` function to compute and save the overall accuracy to the variable `multiclass_accuracy`
- use the `classification_report` function to compute and save the classification metrics for each class to the variable `multiclass_report`

Print the overall accuracy and classification report for our multiclass model.

In [33]:
multiclass_accuracy = accuracy_score(y_test, multiclass_predicted_labels)
multiclass_report = classification_report(y_test, multiclass_predicted_labels)

# show output - do not modify
print(f'Accuracy: {accuracy.item():.4f}')
print(multiclass_report)


Accuracy: 0.8377
              precision    recall  f1-score   support

           0       0.74      0.59      0.66      2158
           1       0.85      0.92      0.89      5798
           2       0.75      0.11      0.19        56

    accuracy                           0.83      8012
   macro avg       0.78      0.54      0.58      8012
weighted avg       0.82      0.83      0.82      8012



Our multiclass neural network performs similarly to the binary classification network at predicting cancellations.

It has an overall accuracy of 84%, meaning that 84% of all the predictions were correct.
The precision in row `1` tells us that when our model predicts a cancellation, it is correct 74% of the time. 
The recall score in row `1` tells us that our model captures 59% of the actual cancellations in our data.

Unfortunately, the model doesn't do the best job of predicting whether or not the customer will no-show. 

For no-shows (row class `2`), the precision score tells us that when our model predicts a no-show it is correct 75% of the time which is surprising well.
However, the low recall score tells us that our model only captures 11% of actual no-shows which is not very good. The lower recall score brings the F1 score down to 19% which indicates a not-so-great balance between precision and recall. This means that the model doesn't predict many no-shows and will most likely not be able to capture most customers who no-show in real-life. 

If our goal is to be able to reach out to potential no-shows, the low recall score is concerning. However, this all may be due to the low number of no-shows in the dataset: it is much harder for our model to find patterns predicting a no-show without more data. However, unlike the binary model, the multiclass does make an attempt to classify no-shows while still being able to predict cancellations ahead of time with similar performance.

So that's the end of our project on predicting hotel cancellations using real-world data! 
In future research, we could improve the model by performing a more in-depth analysis of the features and doing a more robust feature selection process. Some examples might include collecting weather data at the time of each booking, reservations made on major holidays, economic conditions, or even global pandemics and health concerns.

Furthermore, we could also try to improve performance by modifying the neural network architecture like changing the number of nodes across the hidden layers, trying out different activation functions and optimizers, adding more hidden layers, or training on additional epochs, etc.